### 데이터 불러오기

In [1]:
with open("train_set.txt") as f:
    train = f.readlines()
train = str(train)

### 데이터 처리

In [7]:
import pandas as pd

def preprocessing(string):
    df = pd.DataFrame()
    k = string.split("~")
    
    tmp = [] 
    for i in range(1,int((len(k)-4)/3)):
        tmp.append(k[3*i:3*i+4])

    article_id = []
    for i in range(len(tmp)):
        article_id.append(tmp[i][0].split('\\n')[1][4])

    headline = []
    for i in range(len(tmp)):
        h = tmp[i][1].replace('" \\n\', \'','')
        h = h.replace('"','') 
        headline.append(h)

    body = []
    for i in range(len(tmp)):
        body.append(tmp[i][2])

    company_id = []
    for i in range(len(tmp)):
        company_id.append(tmp[i][3].split('\\n')[0])
        
    df['TrainArticleID'] = article_id
    df['Headline'] = headline
    df['Body'] = body
    df['CompanyID'] = company_id
    
    return df

In [35]:
df = preprocessing(train)

In [ ]:
df.head()

In [10]:
df.to_csv("train_df.csv")

### Input Embedding 하기

In [11]:
df = df[:9999]

In [12]:
import nltk
import re
bod = [re.sub('[^A-Za-z ]+', '', df.Body[i]) for i in range(len(df))]
texts = [nltk.word_tokenize(bod[i]) for i in range(len(bod))]

In [13]:
Texts = []
for i in texts:
    tmp = []
    for j in i:
        if len(j)>1:
            tmp.append(j)        
    Texts.append(tmp)

In [36]:
tag = [nltk.pos_tag(texts[i]) for i in range(len(texts))]

In [47]:
with open('word.txt', 'wb') as f:
    pickle.dump(Texts, f)

with open('tag.txt', 'wb') as f:
    pickle.dump(tag, f, pickle.HIGHEST_PROTOCOL) 

### 데이터 불러오기

In [2]:
import pandas as pd
import pickle

df = pd.read_csv("train_df.csv")

In [3]:
del df["Unnamed: 0"]
df.head()

,TrainArticleID,Headline,Body,CompanyID
0,0,Press Release: Dassault Systèmes: Disclosure o...,"""Dassault ODDO & 585 :12:19...",1536
1,1,"Press Release: Home BancShares, Inc. Announces...","""Home BancShares, Inc.Announces Record Net Inc...",2408
2,2,Press Release: Mercury General Corporation -2-,"""-------------------------- \n', 'Net income ...",789
3,3,Press Release: Intelsat Reports Second Quarter...,"""Interest expense, net consists of the gross i...",3330
4,4,ON Semiconductor Files 8K - Other Events >ON,"""ON Semiconductor Corp.(ON) filed a Form 8K - ...",1844


25354

In [103]:
dev = df.loc[0:999]
dev.to_csv('dev.csv')

In [106]:
texts[2]

['Net',
 'income',
 'Net',
 'realized',
 'investmentlosses',
 'gains',
 'net',
 'oftax',
 'Operating',
 'RatiosGAAP',
 'Basis',
 'Loss',
 'ratio',
 'Expense',
 'ratio',
 'Combined',
 'ratio',
 'MERCURY',
 'GENERAL',
 'CORPORATION',
 'AND',
 'SUBSIDIARIESCONDENSED',
 'BALANCE',
 'SHEETS',
 'AND',
 'OTHER',
 'INFORMATIONs',
 'except',
 'pershare',
 'amounts',
 'and',
 'ratiosn',
 'September',
 'December',
 'unaudited',
 'ASSETS',
 'Investments',
 'at',
 'fair',
 'valueFixed',
 'maturity',
 'securitiesamortized',
 'cost',
 'Equity',
 'securities',
 'cost',
 'Shortterm',
 'investmentscost',
 'Total',
 'investments',
 'Cash',
 'ReceivablesPremiums',
 'Accrued',
 'investment',
 'income',
 'Other',
 'Total',
 'receivables',
 'Deferred',
 'policy',
 'acquisitioncosts',
 'Fixed',
 'assets',
 'net',
 'Current',
 'income',
 'taxes',
 'Deferred',
 'income',
 'taxes',
 'Goodwill',
 'Other',
 'intangible',
 'assets',
 'net',
 'Other',
 'assets',
 'Total',
 'assets',
 'LIABILITIES',
 'AND',
 'SHAREHO

In [48]:
with open('word.txt', 'rb') as f:
    texts = pickle.load(f)
with open('tag.txt', 'rb') as f:
    tag = pickle.load(f)

In [5]:
company_map = pd.read_csv('company_map.txt')
company_map.head()

,OfficialName,Ticker,CompanyID
0,INTERSIL CORP-A,ISIL,2094
1,ARMADA HOFFLER PROPERTIES IN,AHH,3399
2,REPLIGEN CORP,RGEN,866
3,REGIS CORP,RGS,867
4,ROYAL GOLD INC,RGLD,868


In [8]:
len(set(company_map.CompanyID)) # 회사수

3751

In [117]:
id_dic = {}
for i in range(len(company_map)):
    id_dic[company_map.CompanyID[i]] = company_map.OfficialName[i]

In [118]:
df["Company_name"]=0

In [ ]:
cid = [df.CompanyID.loc[i].split(",") for i in range(len(df))]
df.CompanyID = cid

In [ ]:
for i in range(len(df)):
    tmp = []
    for j in df.CompanyID.loc[i]:
        tmp.append(id_dic[int(j)])
    df["Company_name"].loc[i] = tmp

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
df.head()

In [ ]:
df[df.Company_name == '461']

In [40]:
df.to_csv("df_1000.csv")

In [52]:
print(texts[0][0])
print(tag[0][0])

Dassault
('Dassault', 'NNP')


In [115]:
df.Headline.loc[3]

'Press Release: Intelsat Reports Second Quarter -2- '

In [ ]:
ne_chunk(tag[i], binary=True)

In [60]:
from nltk import ne_chunk

namedEnt = [ne_chunk(tag[i], binary=True) for i in range(len(tag))]  # NER

In [61]:
with open('chunk.txt', 'wb') as f:
    pickle.dump(namedEnt, f)

In [67]:
a = namedEnt[0]

In [80]:
namedEnt[1] #NE

LookupError: 

===========================================================================
NLTK was unable to find the gs file!
Use software specific configuration paramaters or set the PATH environment variable.
===========================================================================

Tree('S', [Tree('NE', [('Home', 'NN')]), ('BancShares', 'NNS'), ('IncAnnounces', 'VBZ'), Tree('NE', [('Record', 'NNP'), ('Net', 'NNP'), ('Income', 'NNP')]), ('of', 'IN'), Tree('NE', [('Millionn', 'NNP')]), ('CONWAY', 'NNP'), ('Ark', 'NNP'), ('Home', 'NNP'), ('BancShares', 'NNP'), ('IncNASDAQHOMB', 'NNP'), ('parent', 'NN'), ('company', 'NN'), ('of', 'IN'), Tree('NE', [('Centennial', 'NNP'), ('Bank', 'NNP')]), ('today', 'NN'), ('announced', 'VBD'), ('a', 'DT'), ('record', 'NN'), ('quarterly', 'JJ'), ('profit', 'NN'), ('of', 'IN'), ('million', 'CD'), ('for', 'IN'), ('the', 'DT'), ('second', 'JJ'), ('quarter', 'NN'), ('of', 'IN'), ('to', 'TO'), ('million', 'CD'), ('for', 'IN'), ('the', 'DT'), ('same', 'JJ'), ('quarter', 'NN'), ('in', 'IN'), Tree('NE', [('Diluted', 'NNP')]), ('earnings', 'NNS'), ('per', 'IN'), ('share', 'NN'), ('for', 'IN'), ('the', 'DT'), ('second', 'JJ'), ('quarter', 'NN'), ('of', 'IN'), ('per', 'IN'), ('share', 'NN'), ('compared', 'VBN'), ('to', 'TO'), ('per', 'IN'), ('s

In [86]:
sum_tag = sum(tag,[])

In [94]:
nn_tag = []
for i in range(len(sum_tag)):
    if 'NN' in sum_tag[i][1]:
        nn_tag.append(sum_tag[i][0])

In [95]:
nn_tag[:2]

['Dassault', 'ODDO']

In [116]:
df.head()

,TrainArticleID,Headline,Body,CompanyID
0,0,Press Release: Dassault Systèmes: Disclosure o...,"""Dassault ODDO & 585 :12:19...",1536
1,1,"Press Release: Home BancShares, Inc. Announces...","""Home BancShares, Inc.Announces Record Net Inc...",2408
2,2,Press Release: Mercury General Corporation -2-,"""-------------------------- \n', 'Net income ...",789
3,3,Press Release: Intelsat Reports Second Quarter...,"""Interest expense, net consists of the gross i...",3330
4,4,ON Semiconductor Files 8K - Other Events >ON,"""ON Semiconductor Corp.(ON) filed a Form 8K - ...",1844


In [96]:
from gensim.models import Word2Vec
model = Word2Vec(nn_tag, size=100, window=5, min_count=1, workers=4)

In [97]:
model.save("word2vec.model")

In [ ]:
import nltk.data
tagger = nltk.data.load("chunkers/NAME_OF_CHUNKER.pickle")

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# word2vec embedding vector와 word에 대한 tfidf 가중치를 이용한 vectorizer 함수

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
class TfidfEmbeddingVectorizer:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        
    def transform(self, X):
        tfidf = TfidfVectorizer(analyzer = lambda x : x) 
        tfidf.fit(X)
        max_idf = max(tfidf.idf_) 
        word2weight = defaultdict(lambda : max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]) 
        
        array_list =[]
        for words in X:
            array_list.append(np.array(np.mean([self.word2vec[w]*word2weight[w] for w in words if w in self.word2vec] or [np.zeros(300)], axis = 0)))
        return(array_list)

vec_tf = TfidfEmbeddingVectorizer(model)

tf = vec_tf.transform(nn_tag)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [1]:
import numpy as np


# Method to compute the accuracy. Call predict_labels to get the labels for the dataset
def compute_f1(predictions, correct, idx2Label):
    label_pred = []
    for sentence in predictions:
        label_pred.append([idx2Label[element] for element in sentence])

    label_correct = []
    for sentence in correct:
        label_correct.append([idx2Label[element] for element in sentence])

    # print("predictions ", len(label_pred))
    # print("correct labels ", len(label_correct))

    prec = compute_precision(label_pred, label_correct)
    rec = compute_precision(label_correct, label_pred)

    f1 = 0
    if (rec + prec) > 0:
        f1 = 2.0 * prec * rec / (prec + rec);

    return prec, rec, f1


def compute_precision(guessed_sentences, correct_sentences):
    assert (len(guessed_sentences) == len(correct_sentences))
    correctCount = 0
    count = 0

    for sentenceIdx in range(len(guessed_sentences)):
        guessed = guessed_sentences[sentenceIdx]
        correct = correct_sentences[sentenceIdx]
        assert (len(guessed) == len(correct))
        idx = 0
        while idx < len(guessed):
            if guessed[idx][0] == 'B':  # a new chunk starts
                count += 1

                if guessed[idx] == correct[idx]:  # first prediction correct
                    idx += 1
                    correctlyFound = True

                    while idx < len(guessed) and guessed[idx][0] == 'I':  # scan entire chunk
                        if guessed[idx] != correct[idx]:
                            correctlyFound = False 

                        idx += 1

                    if idx < len(guessed):
                        if correct[idx][0] == 'I':  # chunk in correct was longer
                            correctlyFound = False

                    if correctlyFound:
                        correctCount += 1
                else:
                    idx += 1
            else:
                idx += 1

    precision = 0
    if count > 0:
        precision = float(correctCount) / count

    return precision